In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon , shape
import ast
from shapely.ops import transform
from shapely import wkt
from pyproj import CRS, Transformer

from helper_functions import convert_to_list

In [2]:
df=pd.read_json("../input/TN/treenation_new.json")
df.head(2)

,address,created_at,description,id,image,name,project_id,slug,TreeCount,polygon_data
0,,2007-05-01 12:09:33+00:00,La plantation de Dosso est la plantation princ...,1,https://treenation-uploads.s3.eu-central-1.ama...,Dosso Centre forestier,1,dosso-centre-forestier,289959,NaN
1,,2008-02-01 13:10:20+00:00,Pépinière située dans une zone aride. Cette pé...,2,https://treenation-uploads.s3.eu-central-1.ama...,Bonkoukou,1,bonkoukou,0,NaN


### Process geometries

Convert nested lists and swap geometries

In [6]:
df['polygon_data'] = df['polygon_data'].apply(convert_to_list)

Failed to convert string to list: [[42.70258987584701,-121.3658227263177],[42.70133712182819,-121.3671102892821],[42.69980115863795,-121.3689262264679],[42.69952581756378,-121.3702404320167],[42.69917110950835,-121.3716240256548],[42.69841201326724,-121.3719689200832],[42.69790585131582,-121.3729018525551],[42.69765203547103,-121.3747106403452],[42.69782448309881,-121.3756374971106],[42.69721249142377,-121.3787072406694],[42.69670822149998,-121.3812363423232],[42.69630308399198,-121.3822403392514],[42.69483584787033,-121.3843453239961],[42.69275178284798,-121.3870309703975],[42.69173567815716,-121.3893106062852],[42.69151389809912,-121.389650091381],[42.69137860226328,-121.3898571899699],[42.69087950435313,-121.3922534023818],[42.69051213380052,-121.3940947501125],[42.69031239969087,-121.3948039843353],[42.69011744727845,-121.3954962249669],[42.68921222555297,-121.3961759990748],[42.68821980366493,-121.3978627927863],[42.68738074863788,-121.398832265562],[42.6870494599098,-121.39933924

Turn into proper geometries

In [8]:
df['geometry'] = df['polygon_data'].apply(lambda x: Polygon(x) if x is not None and len(x) >= 4 else Polygon())

In [9]:
gdf = gpd.GeoDataFrame(df, geometry = 'geometry', crs = 'EPSG:4326')
gdf['geometry'] = gdf['geometry'].make_valid()

In [10]:
gdf['polygon_areas_km'] = gdf['geometry'].to_crs(3857).area / 1e6

In [11]:
gdf['polygon_areas_km'].describe()

count      655.000000
mean       251.072568
std       3078.758791
min          0.000000
25%          0.017101
50%          0.164769
75%          1.167965
max      74495.153996
Name: polygon_areas_km, dtype: float64

### Preparing columns

In [13]:
columns_rename_mapping = {
    'project_id': 'project_id_reported',
    'TreeCount': 'trees_planted_reported',
    'description': 'description_reported',
    'created_at': 'planting_date_reported',
    'id': 'site_id_reported',
    'image': 'url',
}

gdf.rename(columns=columns_rename_mapping, inplace=True)

In [14]:
gdf.drop(columns=["name", "slug", "address", "polygon_data"], inplace=True)

In [15]:
gdf['host_name'] = 'Tree Nation'

gdf = gdf.assign(species_count_reported=None, species_planted_reported=None,country=None,survival_rate_reported=None)

gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 655 entries, 0 to 654
Data columns (total 13 columns):
 #   Column                    Non-Null Count  Dtype              
---  ------                    --------------  -----              
 0   planting_date_reported    655 non-null    datetime64[ns, UTC]
 1   description_reported      655 non-null    object             
 2   site_id_reported          655 non-null    int64              
 3   url                       655 non-null    object             
 4   project_id_reported       655 non-null    int64              
 5   trees_planted_reported    655 non-null    int64              
 6   geometry                  655 non-null    geometry           
 7   polygon_areas_km          655 non-null    float64            
 8   host_name                 655 non-null    object             
 9   species_count_reported    0 non-null      object             
 10  species_planted_reported  0 non-null      object             
 11  country    

### Save it

In [17]:
gdf.to_file('../input/Plant_for_planet_shapefiles/Tree_Nation.gpkg')